In [1]:
import json
import time
import pandas as pd
import numpy as np
#import requests
from bs4 import BeautifulSoup

from pprint import pprint
from requests_html import HTMLSession, AsyncHTMLSession

In [2]:
BASE_URL = "https://www.tequilamatchmaker.com/tequilas/2328-fortaleza-anejo"

In [3]:
response = requests.get(BASE_URL)

In [4]:
contents = response.content

In [5]:
html = BeautifulSoup(contents, parser = "html.parser")

In [6]:
html

<!DOCTYPE html>
<html>
<head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="width=device-width, initial-scale=1, shrink-to-fit=no" name="viewport"/>
<meta content="https://www.kuinak.com/system/base_product_images/images/000/016/525/detail/fortaleza-anejo-3.jpg?1611764318" property="og:image"/>
<title>Fortaleza Añejo | Tequila Matchmaker</title>
<meta content="authenticity_token" name="csrf-param"/>
<meta content="Lu0GyLWpBcIqLkvhRhz1TISv0enX19OW+GsIil2btAR4Ddpl8flZ3kk2QxTlePyI8A8zgxJ78uWoqoIagL75Ow==" name="csrf-token"/>
<link href="/assets/application-9ca37d6deb8729111f0cbebde783d3255dfbe5c5bf0736cac1ee9fa08cabab34.css" media="all" rel="stylesheet"/>
<link href="https://cdn.jsdelivr.net/npm/instantsearch.js@2.3/dist/instantsearch.min.css" media="screen" rel="stylesheet"/>
<script src="/assets/application-ecd506823bffc0997be33ccef398dd12ecfe020cd70bf801fcc67f5016cbb556.js"></script>
<script src="https://cdn.jsdelivr.net/npm/instantsearch.js@2.3/di

In [324]:
def getTequilaDetails(html):
    name = html.findAll("h1", itemprop = "name")[0].text
    name = name.replace("\n", "")
    
    d = html.findAll("div", itemprop = "description")[0]
    
    ids = d.find_all("th", scope = "row")
    ids = list(map(lambda x: x.text.replace("\n", "").replace(":", ""), ids))
    ids = ["Tequila"] + ids
    
    vals = d.find_all("td")
    vals = list(map(lambda x: x.text.replace("\n", ""), vals))
    vals = list(map(lambda x: x[:-1] if x[-1] == "," else x, vals))
    vals[0] = int(vals[0]) if vals[0].isdigit() else vals[0]
    vals = [name] + vals
    
    
    return dict(zip(ids, vals))

In [270]:
def getCommunityDetails(html, kind="aromas"):
    
    wine_data = html.find_all("div", class_ = f"{kind} section")
    
    flavours = list(
        map(
            lambda x: kind.capitalize() + "_" + x.text,
            wine_data[0].find_all("div", class_ = "name")
        )
    )

    flavours_vals = list(
        map(
            lambda x: x.text.replace("\n", ""),
            wine_data[0].find_all("div", class_ = "taste")
        )
    )
    
    flavours_vals = list(map(lambda x : int(x) if x.isdigit() else x, flavours_vals))
    
    taste_profile = dict(zip(flavours, flavours_vals))
    
    return taste_profile

In [11]:
det = getTequilaDetails(html)

In [12]:
det

{'NOM': 1493,
 'Agave Type': 'Tequilana Weber',
 'Agave Region': 'Jalisco (Tequila Valley)',
 'Region': 'Jalisco (Los Valles)',
 'Cooking': 'Stone/Brick Ovens',
 'Extraction': 'Tahona',
 'Water Source': 'Deep well water',
 'Fermentation': 'Wood fermentation tanks, 100% agave, Open-air fermentation, Fermentation without fibers',
 'Distillation': '2x distilled',
 'Still': 'Copper Pot',
 'Aging': 'American White Oak barrels, Used barrels, Bourbon barrels',
 'ABV/Proof': '40% abv (80-proof)',
 'Other': 'Natural cork, Charcoal filtration, Distilled at or near proof, Blended batches, No additives'}

In [17]:
#wine_data = html.find_all("div", class_ = "col-md-6 col-xl-5")
wine_data = html.find_all("div", class_ = "aromas section")

In [33]:
pprint(wine_data[0])

<div class="aromas section">
<div class="row">
<div class="col-4 col-sm-4 col-md-4 col-lg-3">
<div class="taste">
<div class="count" style="background-color:#825f1c;">146</div>
</div>
<div class="name">Agave (cooked)</div>
</div>
<div class="col-4 col-sm-4 col-md-4 col-lg-3">
<div class="taste">
<div class="count" style="background-color:#e1dabd;">112</div>
</div>
<div class="name">Vanilla</div>
</div>
<div class="col-4 col-sm-4 col-md-4 col-lg-3">
<div class="taste">
<div class="count" style="background-color:#bb8d5e;">103</div>
</div>
<div class="name">Caramel</div>
</div>
<div class="col-4 col-sm-4 col-md-4 col-lg-3">
<div class="taste">
<div class="count" style="background-color:#4f2412;">102</div>
</div>
<div class="name">Oak</div>
</div>
<div class="col-4 col-sm-4 col-md-4 col-lg-3">
<div class="taste">
<div class="count" style="background-color:#dab760;">53</div>
</div>
<div class="name">Butterscotch</div>
</div>
<div class="col-4 col-sm-4 col-md-4 col-lg-3">
<div class="taste">

In [36]:
flavours = list(
    map(
        lambda x: x.text,
        wine_data[0].find_all("div", class_ = "name")
    )
)

In [40]:
flavours_vals = list(
    map(
        lambda x: x.text.replace("\n", ""),
        wine_data[0].find_all("div", class_ = "taste")
    )
)

In [41]:
flavours

['Agave (cooked)',
 'Vanilla',
 'Caramel',
 'Oak',
 'Butterscotch',
 'Pepper (black)',
 'Butter',
 'Cinnamon',
 'Citrus',
 'Earth',
 'Alcohol',
 'Smoke']

In [42]:
flavours_vals

['146', '112', '103', '102', '53', '34', '31', '28', '24', '21', '19', '19']

In [44]:
flavours_vals = list(map(lambda x : int(x) if x.isdigit() else x, flavours_vals))

In [45]:
flavours_vals

[146, 112, 103, 102, 53, 34, 31, 28, 24, 21, 19, 19]

In [142]:
aroma_det = getCommunityDetails(html)

In [143]:
aroma_det

{'Aromas_Agave (cooked)': 146,
 'Aromas_Vanilla': 112,
 'Aromas_Caramel': 103,
 'Aromas_Oak': 102,
 'Aromas_Butterscotch': 53,
 'Aromas_Pepper (black)': 34,
 'Aromas_Butter': 31,
 'Aromas_Cinnamon': 28,
 'Aromas_Citrus': 24,
 'Aromas_Earth': 21,
 'Aromas_Alcohol': 19,
 'Aromas_Smoke': 19}

In [144]:
flavor_det = getCommunityDetails(html, "flavors")

In [145]:
flavor_det

{'Flavors_Agave (cooked)': 163,
 'Flavors_Oak': 114,
 'Flavors_Vanilla': 111,
 'Flavors_Caramel': 97,
 'Flavors_Pepper (black)': 64,
 'Flavors_Butterscotch': 53,
 'Flavors_Butter': 47,
 'Flavors_Cinnamon': 40,
 'Flavors_Citrus': 33,
 'Flavors_Earth': 30,
 'Flavors_Fruity': 24,
 'Flavors_Smoke': 22}

In [271]:
def getReviewerData(review_html, name="Tequila"):
    # Get reviwer name and level of seniority
    
    #print(review_html)
    
    reviewer_name = review_html.find("div", itemprop="author").text.replace("\n", "")
    reviewer_level = review_html.find("div", class_="comment__user-level").text.replace("\n", "")
    
    # How many ratings has reviwer provided on site? This is a signal of trustworthiness and expertise
    num_ratings_given = review_html.find("div", class_="comment__user-ratings").text.replace("\n", " ").strip()
    
    if type(num_ratings_given) == str:
        num_ratings_given = num_ratings_given.split(" ")[0]
        num_ratings_given = int(num_ratings_given) if num_ratings_given.isdigit() else num_ratings_given
    
    # What rating is provided for this tequila?
    rating = review_html.find("li").text.replace("\n", " ").strip().split(" ")[0]
    rating = int(rating) if rating.isdigit() else rating
    
    # The review attached to the rating
    review_text = review_html.find('p', class_="card-text", itemprop="description").text.replace("\n", "")
    review_text = review_text[:-1] if review_text[-1] == " " else review_text
    
    # Does reviewer recommend this tequila?
    recom = review_html.find("span", class_ = "label").text.replace("\n", "")
    recom = 1 if recom.lower() == "i recommend this" else 0
    
    reviewer_data = {
        "Reviewer" : reviewer_name,
        "Reviewer_Level" : reviewer_level,
        "Ratings_Given" : num_ratings_given,
        "Tequila" : name,
        "Rating" : rating,
        "Review" : review_text,
        "Recommend" : recom,
        
    }
    return reviewer_data

In [272]:
def allReviews(html, name="Tequila", start=0):
    review_html = html.find_all("div", itemprop="review")
    
    #print(review_html)
    
    review_dicts = [getReviewerData(el, name) for el in review_html]
    
    data = dict(
        zip(
            range(start, start + len(review_dicts)),
            review_dicts
        )
    )

    return data, start + len(review_dicts)

In [326]:
def tequilaReviews(base_url, idx=0):
    p = 1
    final_reviews = dict()
    soup = 0
    
    html = BeautifulSoup(
            requests.get(base_url).content,
            "html.parser"
        )
    
    name = html.findAll("h1", itemprop = "name")[0].text
    name = name.replace("\n", "")
    
    del(html)
    
    while soup is not None:
        url = base_url.split("-")[0] + f"/reviews?page={p}"
        
        print(f"URL: {url}\n")
        sleep_duration = np.random.randint(low = 2, high=8)
        
        soup = BeautifulSoup(
            requests.get(url).content,
            "html.parser"
        )
        
        if soup.text == "\n":
            break
            
        scraped, idx = allReviews(soup, name, idx)
        
        print(
            f"Extracted review page {p}...\n"
        )
        final_reviews.update(scraped)
        
        print(
        f"Sleeping for {sleep_duration} seconds...\n"
        )
        
        time.sleep(sleep_duration)
        
        p += 1
        
    return final_reviews, idx

In [316]:
def json_to_dataframe(json, expand_len=10):
    
    if type(json) == list:
        all_data = [
            pd.DataFrame(
                data = {
                    k : [d[k] 
                         #for i in range(0, expand_len)
                        ]
                    for k in d
                },
        )
            for d in json
        ]
        
        all_data = pd.concat(all_data, axis=1)
        
    else:    
        if type(json) == str:
            with open(json, "r") as f:
                json = json.load(f)

        all_data = pd.DataFrame(
            data = {k : [json[k]] for k in json},
        )
    return all_data

In [109]:
html.find_all("div", itemprop="review")

[<div class="card comment comment-product-page" itemprop="review" itemscope="" itemtype="http://schema.org/Review">
 <div class="card-header">
 <div class="comment__user text-center">
 <img class="" src="https://www.kuinak.com/system/user_images/images/000/015/872/thumbnail/122071.jpg?1605297090"/>
 <div class="comment__user-name" itemprop="author">
 Mike B (Great Lakes Tequila Club)
 </div>
 <div class="comment__user-level">
 Tequila Phenom
 </div>
 <div class="comment__user-ratings">
 315
 ratings
 </div>
 </div>
 </div>
 <div class="card-block">
 <div class="sr-only" itemprop="reviewRating" itemscope="" itemtype="http://schema.org/Rating">
 <meta content="1" itemprop="worstRating"/>
 <meta content="91" itemprop="ratingValue"/>
 <meta content="100" itemprop="bestRating"/>
 </div>
 <ul class="product-list__item__ratings">
 <li>
 91
 <span class="product-list__item__ratings-label">Rating</span>
 </li>
 </ul>
 <div class="comment__comment">
 <div class="review-of sr-only" itemprop="name

In [195]:
data, _ = allReviews(html)

<div class="card comment comment-product-page" itemprop="review" itemscope="" itemtype="http://schema.org/Review">
<div class="card-header">
<div class="comment__user text-center">
<img class="" src="https://www.kuinak.com/system/user_images/images/000/015/872/thumbnail/122071.jpg?1605297090"/>
<div class="comment__user-name" itemprop="author">
Mike B (Great Lakes Tequila Club)
</div>
<div class="comment__user-level">
Tequila Phenom
</div>
<div class="comment__user-ratings">
315
ratings
</div>
</div>
</div>
<div class="card-block">
<div class="sr-only" itemprop="reviewRating" itemscope="" itemtype="http://schema.org/Rating">
<meta content="1" itemprop="worstRating"/>
<meta content="91" itemprop="ratingValue"/>
<meta content="100" itemprop="bestRating"/>
</div>
<ul class="product-list__item__ratings">
<li>
91
<span class="product-list__item__ratings-label">Rating</span>
</li>
</ul>
<div class="comment__comment">
<div class="review-of sr-only" itemprop="name">
</div>
<p class="card-text"

In [196]:
data

{0: {'Reviewer': 'Mike B (Great Lakes Tequila Club)',
  'Reviewer_Level': 'Tequila Phenom',
  'Ratings_Given': 315,
  'Tequila': 'Fortaleza Añejo',
  'Rating': 91,
  'Review': 'Lot 43|The infamous Lot 43! A friend shared a glass with me and I finally got to try a holy grail tequila. What can I say about this supposedly XA-blended Añejo other than it was delicious aged Fortaleza.',
  'Recommend': 1},
 1: {'Reviewer': 'Jon Novak',
  'Reviewer_Level': 'Tequila Ninja',
  'Ratings_Given': 161,
  'Tequila': 'Fortaleza Añejo',
  'Rating': 91,
  'Review': 'Lot 55|Incredible. Such a dessert sipper. Beautiful barrel notes on the nose and the palate is a treat. From beginning to finish this is a treat through and through.',
  'Recommend': 1},
 2: {'Reviewer': 'Frankie Medina',
  'Reviewer_Level': 'Tequila Savant',
  'Ratings_Given': 287,
  'Tequila': 'Fortaleza Añejo',
  'Rating': 87,
  'Review': 'Lot 63|This lot 63 is one of the better añejo s from Fortaleza since lot 48.',
  'Recommend': 1},
 3

In [197]:
with open("file.json", "w") as f:
    json.dump(data, f)

In [198]:
with open("file.json", "r") as f:
    f = json.load(f)

In [199]:
f

{'0': {'Reviewer': 'Mike B (Great Lakes Tequila Club)',
  'Reviewer_Level': 'Tequila Phenom',
  'Ratings_Given': 315,
  'Tequila': 'Fortaleza Añejo',
  'Rating': 91,
  'Review': 'Lot 43|The infamous Lot 43! A friend shared a glass with me and I finally got to try a holy grail tequila. What can I say about this supposedly XA-blended Añejo other than it was delicious aged Fortaleza.',
  'Recommend': 1},
 '1': {'Reviewer': 'Jon Novak',
  'Reviewer_Level': 'Tequila Ninja',
  'Ratings_Given': 161,
  'Tequila': 'Fortaleza Añejo',
  'Rating': 91,
  'Review': 'Lot 55|Incredible. Such a dessert sipper. Beautiful barrel notes on the nose and the palate is a treat. From beginning to finish this is a treat through and through.',
  'Recommend': 1},
 '2': {'Reviewer': 'Frankie Medina',
  'Reviewer_Level': 'Tequila Savant',
  'Ratings_Given': 287,
  'Tequila': 'Fortaleza Añejo',
  'Rating': 87,
  'Review': 'Lot 63|This lot 63 is one of the better añejo s from Fortaleza since lot 48.',
  'Recommend': 

In [200]:
sample = pd.read_json("file.json").T

In [201]:
sample

,Reviewer,Reviewer_Level,Ratings_Given,Tequila,Rating,Review,Recommend
0,Mike B (Great Lakes Tequila Club),Tequila Phenom,315,Fortaleza Añejo,91,Lot 43|The infamous Lot 43! A friend shared a ...,1
1,Jon Novak,Tequila Ninja,161,Fortaleza Añejo,91,Lot 55|Incredible. Such a dessert sipper. Beau...,1
2,Frankie Medina,Tequila Savant,287,Fortaleza Añejo,87,Lot 63|This lot 63 is one of the better añejo ...,1
3,Jon Novak,Tequila Ninja,161,Fortaleza Añejo,91,Lot 59|Wow! This is worth the price. A total ...,1
4,Nick Salcedo @tequilarankings,Tequila Guru,93,Fortaleza Añejo,91,"On the nose, aromas of dried fig, mixed with a...",1
5,Remy Newland,Tequila Wizard,892,Fortaleza Añejo,90,Lot 60|Nose: A touch of cider and spice. Van...,1
6,Remy Newland,Tequila Wizard,892,Fortaleza Añejo,89,Lot 52|Nose: Light oak sawdust aroma and war...,1
7,Andy Bardon,Tequila Ninja,172,Fortaleza Añejo,92,Lot 42|Lote: 42-AWhat an amazing mistake!Dried...,1
8,Rob Gerard,Tequila Savant,201,Fortaleza Añejo,92,Lot 52|Lote 52 - this is one of my favorite Añ...,1
9,Jeremy Kinney,Tequila Ninja,153,Fortaleza Añejo,94,Lot 43|Super lucky to have gotten my hands on ...,1


In [202]:
aroma_det

{'Aromas_Agave (cooked)': 146,
 'Aromas_Vanilla': 112,
 'Aromas_Caramel': 103,
 'Aromas_Oak': 102,
 'Aromas_Butterscotch': 53,
 'Aromas_Pepper (black)': 34,
 'Aromas_Butter': 31,
 'Aromas_Cinnamon': 28,
 'Aromas_Citrus': 24,
 'Aromas_Earth': 21,
 'Aromas_Alcohol': 19,
 'Aromas_Smoke': 19}

In [203]:
new_sample = pd.concat([sample, json_to_dataframe([aroma_det, flavor_det], expand_len=len(sample))],axis=1)

In [204]:
new_sample.isnull().sum()

Reviewer                  0
Reviewer_Level            0
Ratings_Given             0
Tequila                   0
Rating                    0
Review                    0
Recommend                 0
Aromas_Agave (cooked)     0
Aromas_Vanilla            0
Aromas_Caramel            0
Aromas_Oak                0
Aromas_Butterscotch       0
Aromas_Pepper (black)     0
Aromas_Butter             0
Aromas_Cinnamon           0
Aromas_Citrus             0
Aromas_Earth              0
Aromas_Alcohol            0
Aromas_Smoke              0
Flavors_Agave (cooked)    0
Flavors_Oak               0
Flavors_Vanilla           0
Flavors_Caramel           0
Flavors_Pepper (black)    0
Flavors_Butterscotch      0
Flavors_Butter            0
Flavors_Cinnamon          0
Flavors_Citrus            0
Flavors_Earth             0
Flavors_Fruity            0
Flavors_Smoke             0
dtype: int64

In [205]:
with open("file.json", "w") as f:
    json.dump(data, f)
    
latest_sample = pd.read_json("file.json").T

In [207]:
latest_sample

,Reviewer,Reviewer_Level,Ratings_Given,Tequila,Rating,Review,Recommend
0,Mike B (Great Lakes Tequila Club),Tequila Phenom,315,Fortaleza Añejo,91,Lot 43|The infamous Lot 43! A friend shared a ...,1
1,Jon Novak,Tequila Ninja,161,Fortaleza Añejo,91,Lot 55|Incredible. Such a dessert sipper. Beau...,1
2,Frankie Medina,Tequila Savant,287,Fortaleza Añejo,87,Lot 63|This lot 63 is one of the better añejo ...,1
3,Jon Novak,Tequila Ninja,161,Fortaleza Añejo,91,Lot 59|Wow! This is worth the price. A total ...,1
4,Nick Salcedo @tequilarankings,Tequila Guru,93,Fortaleza Añejo,91,"On the nose, aromas of dried fig, mixed with a...",1
5,Remy Newland,Tequila Wizard,892,Fortaleza Añejo,90,Lot 60|Nose: A touch of cider and spice. Van...,1
6,Remy Newland,Tequila Wizard,892,Fortaleza Añejo,89,Lot 52|Nose: Light oak sawdust aroma and war...,1
7,Andy Bardon,Tequila Ninja,172,Fortaleza Añejo,92,Lot 42|Lote: 42-AWhat an amazing mistake!Dried...,1
8,Rob Gerard,Tequila Savant,201,Fortaleza Añejo,92,Lot 52|Lote 52 - this is one of my favorite Añ...,1
9,Jeremy Kinney,Tequila Ninja,153,Fortaleza Añejo,94,Lot 43|Super lucky to have gotten my hands on ...,1


In [209]:
new_sample.to_excel("sample.xlsx", index=False)

In [210]:
new_site = "https://www.tequilamatchmaker.com/tequilas/7275-volans-extra-anejo-6-year"
r = requests.get(new_site)

soup = BeautifulSoup(r.content, "html.parser")

In [211]:
all_rev, _ = allReviews(soup, _)

<div class="card comment comment-product-page" itemprop="review" itemscope="" itemtype="http://schema.org/Review">
<div class="card-header">
<div class="comment__user text-center">
<img class="" src="https://www.kuinak.com/system/user_images/images/000/014/883/thumbnail/155966.jpg?1594981518"/>
<div class="comment__user-name" itemprop="author">
Greg Bartolotta @aged_agave
</div>
<div class="comment__user-level">
Tequila Phenom
</div>
<div class="comment__user-ratings">
361
ratings
</div>
</div>
</div>
<div class="card-block">
<div class="sr-only" itemprop="reviewRating" itemscope="" itemtype="http://schema.org/Rating">
<meta content="1" itemprop="worstRating"/>
<meta content="89" itemprop="ratingValue"/>
<meta content="100" itemprop="bestRating"/>
</div>
<ul class="product-list__item__ratings">
<li>
89
<span class="product-list__item__ratings-label">Rating</span>
</li>
</ul>
<div class="comment__comment">
<div class="review-of sr-only" itemprop="name">
</div>
<p class="card-text" itemp

In [212]:
all_rev

{25: {'Reviewer': 'Greg Bartolotta @aged_agave',
  'Reviewer_Level': 'Tequila Phenom',
  'Ratings_Given': 361,
  'Tequila': 'Volans Extra Añejo (6 Year)',
  'Rating': 89,
  'Review': 'Amazing nose, light in color yet rich in flavors of spice, cooked agave and light barrel notes. This tequila took over 13 yrs to get to market. It is costly but special.',
  'Recommend': 1},
 26: {'Reviewer': 'Doug Price',
  'Reviewer_Level': 'Tequila Phenom',
  'Ratings_Given': 312,
  'Tequila': 'Volans Extra Añejo (6 Year)',
  'Rating': 93,
  'Review': 'What a treat to taste this brand new 6 year XA from Volans. It’s pricey but they started this project over 6 years ago when the brand first formed. If your budget allows for it, this should be in your collection. It takes the original XA to new layers of depth with a little higher ABV. Congrats Volans. Another masterpiece from El Pandillo and Volans.',
  'Recommend': 1},
 27: {'Reviewer': 'Gregory Agave Spectator',
  'Reviewer_Level': 'Tequila Aficionado

In [213]:
with open("file.json", "a") as f:
    json.dump(all_rev, f)

In [275]:
def completeDrinkProfile(html):
    total_profile = getTequilaDetails(html)
    total_profile.update(getCommunityDetails(html, "aromas"))
    total_profile.update(getCommunityDetails(html, "flavors"))
    
    return json_to_dataframe(total_profile)

In [238]:
ds = [completeDrinkProfile(h) for h in [html, soup]]

In [249]:
tequila_profile = pd.concat(ds, axis=0).reset_index(drop=True)

In [250]:
tequila_profile.isnull().sum()

Tequila                   0
NOM                       0
Agave Type                0
Agave Region              0
Region                    0
Cooking                   0
Extraction                0
Water Source              0
Fermentation              0
Distillation              0
Still                     0
Aging                     0
ABV/Proof                 0
Other                     0
Aromas_Agave (cooked)     0
Aromas_Vanilla            0
Aromas_Caramel            0
Aromas_Oak                0
Aromas_Butterscotch       1
Aromas_Pepper (black)     0
Aromas_Butter             0
Aromas_Cinnamon           1
Aromas_Citrus             1
Aromas_Earth              1
Aromas_Alcohol            1
Aromas_Smoke              1
Flavors_Agave (cooked)    0
Flavors_Oak               0
Flavors_Vanilla           0
Flavors_Caramel           0
Flavors_Pepper (black)    0
Flavors_Butterscotch      1
Flavors_Butter            1
Flavors_Cinnamon          1
Flavors_Citrus            1
Flavors_Earth       

In [236]:
getTequilaDetails(soup)

{'Tequila': 'Volans Extra Añejo (6 Year)',
 'NOM': 1579,
 'Agave Type': 'Tequilana Weber',
 'Agave Region': 'Jalisco (Los Altos)',
 'Region': 'Jalisco (Los Altos Southern)',
 'Cooking': 'Stone/Brick Ovens',
 'Extraction': 'Tahona',
 'Water Source': 'Deep well water',
 'Fermentation': 'Stainless steel tanks, 100% agave, Open-air fermentation, Fermentation without fibers',
 'Distillation': '2x distilled',
 'Still': 'Copper Pot',
 'Aging': 'Whisky barrels, Used barrels',
 'ABV/Proof': '43% abv (86-proof)',
 'Other': 'No additives'}

In [252]:
sample.to_excel("reviewer-sample.xlsx", index=False)
tequila_profile.to_excel("tequila-profile-sample.xlsx", index=False)

In [253]:
tequila_profile.head()

,Tequila,NOM,Agave Type,Agave Region,Region,Cooking,Extraction,Water Source,Fermentation,Distillation,...,Aromas_Banana,Aromas_Fruity,Aromas_Pineapple,Aromas_Grapes,Flavors_Minerals,Flavors_Grass,Flavors_Grass (cut),Flavors_Vegetal,Flavors_Banana,Flavors_Grapes
0,Fortaleza Añejo,1493,Tequilana Weber,Jalisco (Tequila Valley),Jalisco (Los Valles),Stone/Brick Ovens,Tahona,Deep well water,"Wood fermentation tanks, 100% agave, Open-air ...",2x distilled,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Volans Extra Añejo (6 Year),1579,Tequilana Weber,Jalisco (Los Altos),Jalisco (Los Altos Southern),Stone/Brick Ovens,Tahona,Deep well water,"Stainless steel tanks, 100% agave, Open-air fe...",2x distilled,...,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0


In [254]:
tequila_profile.to_json("tequila-profile-sample.json", orient="index")

In [314]:
all_ = tequilaReviews(BASE_URL)

URL: https://www.tequilamatchmaker.com/tequilas/2328/reviews?page=1






Mike B (Great Lakes Tequila Club)


Tequila Phenom


316
ratings











91
Rating






Lot 43
|
The infamous Lot 43! A friend shared a glass with me and I finally got to try a holy grail tequila. What can I say about this supposedly XA-blended Añejo other than it was delicious aged Fortaleza. 



I recommend this

about 1 month ago









Jon Novak


Tequila Ninja


161
ratings











91
Rating






Lot 55
|
Incredible. Such a dessert sipper. Beautiful barrel notes on the nose and the palate is a treat. From beginning to finish this is a treat through and through. 



I recommend this

about 2 months ago









Frankie Medina


Tequila Savant


287
ratings











87
Rating






Lot 63
|
This lot 63 is one of the better añejo s from Fortaleza since lot 48.



I recommend this

3 months ago









Jon Novak


Tequila Ninja


161
ratings











91
Rating






Lot 59
|
Wow! This is worth the

URL: https://www.tequilamatchmaker.com/tequilas/2328/reviews?page=2






Scarlet Sanschagrin


Tequila Boss


539
ratings











89
Rating






Lot 46
|
Lighter than the normal anejo, but has a richer mouthfeel and a cheesier aroma than we usually get in the anejo. After it opens, surprising honey aroma in the bottom of the glass. (New barrels?). A little different but good!



I recommend this

almost 4 years ago









Ralph Jimenez


Tequila Ninja


199
ratings











91
Rating






Lot 28
|
Aroma had loads of caramel, baked agave, and some vanilla, along with oak. Flavor has plenty of sweet baked agave, caramel with a hint of wood. The finish has a medium mouth feel with a great long finish. 



I recommend this

almost 4 years ago









Seattle Juan


Tequila Honey Badger


1277
ratings











92
Rating






Lot 21
|
Opened for my birthday. Creamy caramelized agave, barrel notes of oak and vanilla and a wonderful finish. Salud to being 66 years old.



I recom

URL: https://www.tequilamatchmaker.com/tequilas/2328/reviews?page=3






Tommy Grules - Los Skeletone Blues Band 


Tequila Ninja


196
ratings











87
Rating






Lot 35
|
Lot 35. Lot 35. 



I recommend this

over 6 years ago









Tommy Grules - Los Skeletone Blues Band 


Tequila Ninja


196
ratings











90
Rating






Lot 34
|
Lot 34. 



I recommend this

almost 7 years ago









Tommy Grules - Los Skeletone Blues Band 


Tequila Ninja


196
ratings











91
Rating






Lot 29
|
Lot 29. Always expect good things from Fortaleza and this does not disappoint... Oily mouthfeel , smooth and a tad mellower compared to lot 25!... 



I recommend this

about 7 years ago









Rambo Serrato


Tequila Savant


207
ratings











94
Rating






of the 3, this is bold leader.  the man of the house. but he wears custom suits, drives a jag, and loves the ladies. 

not tough. ... experienced and smart. 

big vanilla on the nose.  sweet, inviting,  not at all i

URL: https://www.tequilamatchmaker.com/tequilas/2328/reviews?page=5






Chad C


Tequila Phenom


361
ratings











92
Rating






Lot 49
|
Fortaleza night! The first of three batches! Delicious!



I recommend this

4 months ago









Andrew Bishop


Tequila Connoisseur


74
ratings











93
Rating






Lot 63
|
Notably oakier than lot 61 (my personal favorite), but still a remarkable bottle.



I recommend this

5 months ago









Steve Lere


Tequila Ninja


145
ratings











91
Rating






Lot 63
|
lote 63



I recommend this

5 months ago









christine h


Tequila Savant


202
ratings











92
Rating






Lot 43
|
Fortaleza does not disappoint. I have had several lotes of this over the last 10 or so years but didn't know to keep track. 

Nose: Sweet agave maple banana a little oak some butterscotch vanilla barrel spice 

Palate: agave oak vanilla a little pepper/heat in a good way 

Finish: medium/long, warms the throat 

Nice legs, very well-b

URL: https://www.tequilamatchmaker.com/tequilas/2328/reviews?page=7






Scot Stetson 


Tequila Savant


283
ratings











90
Rating






Rich & creamy. Lots of agave,  raw & cooked. A spicy, citrus, carmel in the taste. Finishes with butter, barrel spices and toffee. 



I recommend this

about 1 year ago









Frank Galvez


Tequila Guru


88
ratings











91
Rating






Lot 59
|
Fortaleza Añejo
the nose is a great blend of cooked agave,  great notes of oak barrel, vanilla, candy yams, kiote, minerals, raisins, dates, fruit cake
the palate is a delight all the great flavors of the Instaglam fancy bottle stuff minus the additives and the awesome minerals. the oak, vanilla, papaya, banana dessert flavors are a blast.
the finish makes me feel like a just smoked a montecristo. soo complex..im big on high proof blancos. considering that, this is the best añejo I've had, to date .



I recommend this

about 1 year ago









Daniela Pedraza


Tequila Fan


3
ratings






URL: https://www.tequilamatchmaker.com/tequilas/2328/reviews?page=8






Richard  Gonzalez 


Tequila Fan


1
ratings











85
Rating






Lot 52
|
great, sweet aroma. lots of butterscotch. quite a bit of oak on the palatte



I recommend this

almost 2 years ago









Kristen Dhanekula


Tequila Enthusiast


6
ratings











89
Rating






Smooth, rich, & complex sip. There's a buttery richness up front and subtle cinnamon & banana end notes. 



I recommend this

about 2 years ago









TJ Matlack


Tequila Fanatic


17
ratings











88
Rating






Lot 52
|
Vanilla and buttery nose. Agave forward flavor. Slight earthiness. Light and refreshing. 



I recommend this

about 2 years ago









Jace LeBrock


Tequila Enthusiast


5
ratings











100
Rating






very hard for me to get my hands ... obviously I was very excited! very clean oak barrel nose...Like smelling delicious pipe tobacco as a child. You know this is going to be a pleasurable drink base

URL: https://www.tequilamatchmaker.com/tequilas/2328/reviews?page=9






Joe Barraza


Tequila Fanatic


16
ratings











95
Rating






Lot 49
|
very good tequila. definitely alot better than reposado, although the repo is still good. I personally prefer Anejo over repo's. still a burn at first not to bad and the after taste is very good. very well made



I recommend this

over 2 years ago









Tom Spangler


Tequila Maniac


30
ratings











95
Rating






Lot 43
|
Fantastic...absolutely love this Tequila 



I recommend this

almost 3 years ago









Eazy Breezy


Tequila Guru


75
ratings











91
Rating






Aroma:  cooked agave, pineapple, vanilla




I recommend this

almost 3 years ago









Harlan Burhans


Tequila Lover


14
ratings











94
Rating






Happy Birthday to me... Love you Missy



I recommend this

almost 3 years ago









Chris S


Tequila Maniac


32
ratings











91
Rating






Lot 45
|
Agave is still there, amazin

URL: https://www.tequilamatchmaker.com/tequilas/2328/reviews?page=11






Sarah Newland


Tequila Warrior


41
ratings











93
Rating






Lot 42
|
Beach wood, vanilla, ripe cantaloupe, butterscotch, Carmel, cinnamon, vanilla ice cream, warm sunbaked wood, coconut, oak, banana, baked sugar, 



I recommend this

almost 4 years ago









Sarah Newland


Tequila Warrior


41
ratings











90
Rating






Lot 41
|
Butterscotch, bourbon, minerals, cream, cinnamon, dry soil, aged wood, 



I recommend this

almost 4 years ago









Stan B


Tequila Freak


24
ratings











80
Rating






Tastes really floral and grapefruit-ty. 



I recommend this

almost 4 years ago









Sean Aldrich


Tequila Ninja


148
ratings











94
Rating






Lot 42
|
Everything is perfect .my highest rated tequila..Lote 42-A....



I recommend this

about 4 years ago









Jeff Hines


Tequila Guru


95
ratings











92
Rating






Lot 43A is an Anejo but contains XA jui

URL: https://www.tequilamatchmaker.com/tequilas/2328/reviews?page=13






Mark Petelin


Tequila Fan


2
ratings











83
Rating






Lot 34
|
Sweet smell
Light burn at first



I recommend this

about 6 years ago









Gary Laspa


Tequila Aficionado


52
ratings











98
Rating






Lot 34
|
OMG.  This is like candy in a glass.  Barely sweet, but simply delicious.  Hint of maple syrup and butterscotch on the nose.  The flavour is butterscotch too, with just a hint of bite, a little smoke, but little of the agave from the Reposado.  



I recommend this

over 6 years ago









Carmen Cox


Tequila Warrior


49
ratings











90
Rating






I can't buy where I'm from so had to try while in California!  So smooth and tasty but pricey. Worth it if you are looking for a high end tequila 




I recommend this

over 6 years ago









Michael  Sasse


Tequila Fan


1
ratings











98
Rating






Lot 25
|
Lot 25. 



I recommend this

over 6 years ago










URL: https://www.tequilamatchmaker.com/tequilas/2328/reviews?page=15






Frank Pasinski


Tequila Fan


2
ratings











78
Rating






Best tequila for the money out there. Reposado and anejo are both excellent. Always have a bottle



I recommend this

almost 10 years ago









Joe Spelman


Tequila Lover


11
ratings











98
Rating






Agave restaurant in NYC a must stop. see Jay



I recommend this

about 10 years ago









shekinah shephard


Tequila Fan


2
ratings











87
Rating






My favorite



I recommend this

almost 11 years ago






Extracted review page 15...

Sleeping for 4 seconds...

URL: https://www.tequilamatchmaker.com/tequilas/2328/reviews?page=16





In [315]:
all_

{0: {'Reviewer': 'Mike B (Great Lakes Tequila Club)',
  'Reviewer_Level': 'Tequila Phenom',
  'Ratings_Given': 316,
  'Tequila': 'Fortaleza Añejo',
  'Rating': 91,
  'Review': 'Lot 43|The infamous Lot 43! A friend shared a glass with me and I finally got to try a holy grail tequila. What can I say about this supposedly XA-blended Añejo other than it was delicious aged Fortaleza.',
  'Recommend': 1},
 1: {'Reviewer': 'Jon Novak',
  'Reviewer_Level': 'Tequila Ninja',
  'Ratings_Given': 161,
  'Tequila': 'Fortaleza Añejo',
  'Rating': 91,
  'Review': 'Lot 55|Incredible. Such a dessert sipper. Beautiful barrel notes on the nose and the palate is a treat. From beginning to finish this is a treat through and through.',
  'Recommend': 1},
 2: {'Reviewer': 'Frankie Medina',
  'Reviewer_Level': 'Tequila Savant',
  'Ratings_Given': 287,
  'Tequila': 'Fortaleza Añejo',
  'Rating': 87,
  'Review': 'Lot 63|This lot 63 is one of the better añejo s from Fortaleza since lot 48.',
  'Recommend': 1},
 3

In [325]:
def getAllData(base_url, idx):
    html = requests.get(base_url)
    html = BeautifulSoup(html.content, "html.parser")
    
    tequila_details = completeDrinkProfile(html)
    reviews, idx = tequilaReviews(base_url, idx)
    
    return tequila_details, reviews

In [3]:
class Obtainer():
    def __init__(self, base_url, idx):
        self.idx = idx if idx else 0
        self.base_url = base_url
        
        self.base_html = BeautifulSoup(
            requests.get(self.base_url).content,
            "html.parser"
            )
        
        self.tequilaName = self.base_html.findAll("h1", itemprop = "name")[0].text.replace("\n", "")
        
    def run(self):
        self.reviewerData = self.tequilaReviews()
        self.tequilaData = self.getTequilaDetails()
        
        self.communityData = self.getCommunityDetails(kind="aromas")
        self.communityData.update(self.getCommunityDetails(kind="flavors"))
        
        print(
            f"Site for {self.tequilaName} scraped!\n"
        )
        
        return
        
        
    def getReviewerData(self, review_html, name="Tequila"):
        """Pick out the needed data fields from a review."""
        # Get reviwer name and level of seniority

        reviewer_name = review_html.find("div", itemprop="author").text.replace("\n", "")
        reviewer_level = review_html.find("div", class_="comment__user-level").text.replace("\n", "")

        # How many ratings has reviwer provided on site? This is a signal of trustworthiness and expertise
        num_ratings_given = review_html.find("div", class_="comment__user-ratings").text.replace("\n", " ").strip()

        if type(num_ratings_given) == str:
            num_ratings_given = num_ratings_given.split(" ")[0]
            num_ratings_given = int(num_ratings_given) if num_ratings_given.isdigit() else num_ratings_given

        # What rating is provided for this tequila?
        rating = review_html.find("li").text.replace("\n", " ").strip().split(" ")[0]
        rating = int(rating) if rating.isdigit() else rating

        # The review attached to the rating
        review_text = review_html.find('p', class_="card-text", itemprop="description").text.replace("\n", "")
        review_text = review_text[:-1] if review_text[-1] == " " else review_text

        # Does reviewer recommend this tequila?
        recom = review_html.find("span", class_ = "label").text.replace("\n", "")
        recom = 1 if recom.lower() == "i recommend this" else 0

        reviewer_data = {
            "Reviewer" : reviewer_name,
            "Reviewer_Level" : reviewer_level,
            "Ratings_Given" : num_ratings_given,
            "Tequila" : name,
            "Rating" : rating,
            "Review" : review_text,
            "Recommend" : recom,

        }
        return reviewer_data
    
    def allReviews(self, html, name="Tequila"):
        """Pick out required data fields for all reviews on a page."""
        
        review_html = html.find_all("div", itemprop="review")

        #print(review_html)

        review_dicts = [self.getReviewerData(el, name) for el in review_html]

        data = dict(
            zip(
                range(self.idx, self.idx + len(review_dicts)),
                review_dicts
            )
        )

        return data, self.idx + len(review_dicts)
    
    
    def tequilaReviews(self,):
        """Pick out required data fields for all reviews, for all review pages."""
        
        p = 1
        final_reviews = dict()
        soup = 0

        while soup is not None:
            url = self.base_url.split("-")[0] + f"/reviews?page={p}"

            print(f"URL: {url}\n")
            sleep_duration = np.random.randint(low = 2, high=8)

            soup = BeautifulSoup(
                requests.get(url).content,
                "html.parser"
            )

            if soup.text == "\n":
                break

            scraped, self.idx = self.allReviews(soup, self.tequilaName)

            print(
                f"Extracted review page {p}...\n"
            )
            final_reviews.update(scraped)

            print(
            f"Sleeping for {sleep_duration} seconds...\n"
            )

            time.sleep(sleep_duration)

            p += 1

        return final_reviews
    
    
    def getTequilaDetails(self):
        """Pick out tequila bottle chracteristics."""

        table = self.base_html.findAll("div", itemprop = "description")[0]

        ids = table.find_all("th", scope = "row")
        ids = list(map(lambda x: x.text.replace("\n", "").replace(":", ""), ids))
        ids = ["Tequila"] + ids

        vals = table.find_all("td")
        vals = list(map(lambda x: x.text.replace("\n", ""), vals))
        vals = list(map(lambda x: x[:-1] if x[-1] == "," else x, vals))
        
        vals[0] = int(vals[0]) if vals[0].isdigit() else vals[0]
        vals = [self.tequilaName] + vals

        return dict(zip(ids, vals))
    
    
    def getCommunityDetails(self, kind="aromas"):

        tequila_data = self.base_html.find_all("div", class_ = f"{kind} section")

        flavours = list(
            map(
                lambda x: kind.capitalize() + "_" + x.text,
                tequila_data[0].find_all("div", class_ = "name")
            )
        )

        flavours_vals = list(
            map(
                lambda x: x.text.replace("\n", ""),
                tequila_data[0].find_all("div", class_ = "taste")
            )
        )

        flavours_vals = list(map(lambda x : int(x) if x.isdigit() else x, flavours_vals))

        taste_profile = dict(zip(flavours, flavours_vals))

        return taste_profile
    
    def clear(self):
        self.reviewerData.clear()
        self.tequilaData.clear()
        
        self.communityData.clear()
        
        print(
            "All cached data cleared!\n"
        )
        return

In [8]:
obt = Obtainer(BASE_URL, idx=0)

In [9]:
obt.run()

URL: https://www.tequilamatchmaker.com/tequilas/2328/reviews?page=1

Extracted review page 1...

Sleeping for 7 seconds...

URL: https://www.tequilamatchmaker.com/tequilas/2328/reviews?page=2

Extracted review page 2...

Sleeping for 6 seconds...

URL: https://www.tequilamatchmaker.com/tequilas/2328/reviews?page=3

Extracted review page 3...

Sleeping for 4 seconds...

URL: https://www.tequilamatchmaker.com/tequilas/2328/reviews?page=4

Extracted review page 4...

Sleeping for 6 seconds...

URL: https://www.tequilamatchmaker.com/tequilas/2328/reviews?page=5

Extracted review page 5...

Sleeping for 3 seconds...

URL: https://www.tequilamatchmaker.com/tequilas/2328/reviews?page=6

Extracted review page 6...

Sleeping for 7 seconds...

URL: https://www.tequilamatchmaker.com/tequilas/2328/reviews?page=7

Extracted review page 7...

Sleeping for 7 seconds...

URL: https://www.tequilamatchmaker.com/tequilas/2328/reviews?page=8

Extracted review page 8...

Sleeping for 5 seconds...

URL: htt

In [22]:
obt.communityData

{}

In [21]:
obt.reviewerData

{}

In [19]:
obt.idx

353

In [ ]:
class WebCrawler():
    def __init__(self, idx=None, sites=None):
        self.idx = idx if bool(idx) is True else 0
        self.sites = sites
        self.obtainer = Obtainer()
        
    def getAllData(base_url, idx):
        html = requests.get(base_url)
        html = BeautifulSoup(html.content, "html.parser")

        tequila_details = self.completeDrinkProfile(html)
        reviews, idx = tequilaReviews(base_url, idx)

        return tequila_details, reviews
    
    def completeDrinkProfile(self):
        total_profile = self.obtainer.getTequilaDetails()
        total_profile.update(self.obtainer.getCommunityDetails(html, "aromas"))
        total_profile.update(getCommunityDetails(html, "flavors"))

        return json_to_dataframe(total_profile)

    def json_to_dataframe(json, expand_len=10):

        if type(json) == list:
            all_data = [
                pd.DataFrame(
                    data = {
                        k : [d[k] 
                             #for i in range(0, expand_len)
                            ]
                        for k in d
                    },
            )
                for d in json
            ]

            all_data = pd.concat(all_data, axis=1)

        else:    
            if type(json) == str:
                with open(json, "r") as f:
                    json = json.load(f)

            all_data = pd.DataFrame(
                data = {k : [json[k]] for k in json},
            )
        return all_data

In [ ]:
class MegaObtainer(Obtainer):
    def __init__(
        self,
        page_num=0,
        urls=None
    ):
        if urls is None:
            self.page_num = page_num
            self.pattern = "https://www.tequilamatchmaker.com/tequilas?q=&hPP=30&idx=BaseProduct&p={}&fR[spirit][0]=Tequila",
            self.url = self.pattern.format(self.page_num)
            
        self.reviewerData = self.tequilaReviews()
        self.tequilaData = self.getTequilaDetails()
        self.communityData = dict()
        self.communityData.update(self.obtainer.getCommunityDetails())
        tequilaData
        
        self.urls = urls
        self.obtainers = list()
        self.obtainer = Obtainer()
        
    def getChildren(self):
        
        return children
    
    def commit(self, path = None, mode = "w", review_fname="tequila-reviews.json", tequila_fname = "tequila-details.json"):
        
        if path is not None:
            review_fname = os.path.join(path, review_fname)
            tequila_fname = os.path.join(path, tequila_fname)
        
        # Dump reviewer data to file
        with open(review_fname, mode) as f:
            json.dump(self.obtainer.reviewerData, f)
            f.close()
        # Dump tequila data
        with open(tequila_fname, mode) as f:
            json.dump(self.obtainer.communityData, f)
            
        # Clear out obtainer cache
        self.obtainer.clear()
        
        return
    
    

In [ ]:
class WebCrawler():
    def __init__(self, sites=None):
        self.sites = sites
        
    def getAllData(base_url, idx):
        html = requests.get(base_url)
        html = BeautifulSoup(html.content, "html.parser")

        tequila_details = completeDrinkProfile(html)
        reviews, idx = tequilaReviews(base_url, idx)

        return tequila_details, reviews
    
    def completeDrinkProfile(html):
        total_profile = getTequilaDetails(html)
        total_profile.update(getCommunityDetails(html, "aromas"))
        total_profile.update(getCommunityDetails(html, "flavors"))

        return json_to_dataframe(total_profile)

    def json_to_dataframe(json, expand_len=10):

        if type(json) == list:
            all_data = [
                pd.DataFrame(
                    data = {
                        k : [d[k] 
                             #for i in range(0, expand_len)
                            ]
                        for k in d
                    },
            )
                for d in json
            ]

            all_data = pd.concat(all_data, axis=1)

        else:    
            if type(json) == str:
                with open(json, "r") as f:
                    json = json.load(f)

            all_data = pd.DataFrame(
                data = {k : [json[k]] for k in json},
            )
        return all_data

    def tequilaReviews(base_url, idx=0):
        p = 1
        final_reviews = dict()
        soup = 0

        html = BeautifulSoup(
                requests.get(base_url).content,
                "html.parser"
            )

        name = html.findAll("h1", itemprop = "name")[0].text
        name = name.replace("\n", "")

        del(html)

        while soup is not None:
            url = base_url.split("-")[0] + f"/reviews?page={p}"

            print(f"URL: {url}\n")
            sleep_duration = np.random.randint(low = 2, high=8)

            soup = BeautifulSoup(
                requests.get(url).content,
                "html.parser"
            )

            if soup.text == "\n":
                break

            scraped, idx = allReviews(soup, name, idx)

            print(
                f"Extracted review page {p}...\n"
            )
            final_reviews.update(scraped)

            print(
            f"Sleeping for {sleep_duration} seconds...\n"
            )

            time.sleep(sleep_duration)

            p += 1

        return final_reviews, idx

In [4]:
s = 'https://www.tequilamatchmaker.com/tequilas?q=&hPP=30&idx=BaseProduct&p=1&fR[spirit][0]=Tequila'

In [8]:
sess = AsyncHTMLSession()

In [9]:
r = await sess.get(s)

In [15]:
r.html.render(sleep=1)

RuntimeError: This event loop is already running